# Road Safety in San Diego County

# Overview

# Names

# Group Member IDs

# Research Question

# Background and Prior Work


Fill in your background and prior work here

References (include links):

1)
2)

# Hypothesis

# Datasets

Fill in your dataset information here

(Copy this information for each dataset)

Dataset Name:
Link to the dataset:
Number of observations:
1-2 sentences describing each dataset.

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
# !pip install geojson

In [2]:
#imports 

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For data cleaning
import string

# Display plots directly in the notebook instead of in a new window
%matplotlib inline

In [3]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

# Data Cleaning

In [4]:
# Read police stops Data Frame
df_police_stops = pd.read_csv("datasets/police_stops/ripa_stops_datasd_v1.csv")

# Read collisions Data Frame
df_collisions = pd.read_csv("datasets/traffic_collisions/pd_collisions_datasd_v1.csv")

# Read police calls in 2015, 2016, 2017, 2018, and 2019 into seperate Data Frames
# df_police_calls1 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2015_datasd_v1.csv")
# df_police_calls2 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2016_datasd_v1.csv")
# df_police_calls3 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2017_datasd_v1.csv")
# df_police_calls4 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2018_datasd.csv")
# df_police_calls5 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2019_datasd.csv")
df_police_calls = pd.read_csv("datasets/police_calls/pd_calls_for_service_all.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Merging police calls in 2015,2016,2017,2018,2019 into one Data Frame 
# df_police_calls = pd.concat([df_police_calls1, df_police_calls2,df_police_calls3,df_police_calls4,df_police_calls5])

# check that the Data Frames were merged properly
# df_police_calls.head()  # shows 2015 data
# # df_police_calls.tail() # shows 2019 data

# wrote a dataset with all police calls 
# df_police_calls.to_csv("datasets/police_calls/pd_calls_for_service_all.csv", index=False)

# DATA CLEANING

## df_collisions

In [6]:
# Traffic Collisions

# Returns the current structure, shape, column names, and data type of each column
# df_collisions.info()

In [7]:
# Traffic Collisions

# Date Time

# Creating seperate columns for Date and Time
date_time = df_collisions['date_time'].str.split(' ', n=1, expand=True)
df_collisions = df_collisions.drop(['date_time'], axis=1)
df_collisions['date'] = date_time[0]
df_collisions['time'] = date_time[1]


In [8]:

# Convert the 'date' and 'time' columns to datetime data types 
df_collisions['date']= pd.to_datetime(df_collisions['date']) 
df_collisions['time']= pd.to_datetime(df_collisions['time'])

# Checking that columns 'date' and 'time' are now datetime objects

# assert isinstance(df_collisions.date,datetime)

In [9]:
# Traffic Collisions

# droping useless columns
drop_cols_collision = ['address_pd_primary','address_pd_intersecting', 'address_name_intersecting', 'address_sfx_intersecting','report_id']
df_collisions = df_collisions.drop(drop_cols_collision, axis = 1)


In [10]:
# Traffic Collisions


# Changing datatype of address_number_primary from int to str
df_collisions['address_number_primary'] = df_collisions.address_number_primary.astype(str)

In [11]:
# Traffic Collisions

# String manipulations

# Making everything lowercase
df_collisions['address_road_primary'] = df_collisions['address_road_primary'].str.lower()
df_collisions['address_sfx_primary'] = df_collisions['address_sfx_primary'].str.lower()
df_collisions['charge_desc'] = df_collisions['charge_desc'].str.lower()
df_collisions['hit_run_lvl'] = df_collisions['hit_run_lvl'].str.lower()

#stripping gets rid of extraneous spaces to allow for easier processing of data
df_collisions['address_road_primary'] = df_collisions.address_road_primary.str.strip()
df_collisions['address_sfx_primary'] = df_collisions.address_sfx_primary.str.strip()
df_collisions['violation_section']= df_collisions.violation_section.str.strip()
df_collisions['violation_type'] = df_collisions.violation_type.str.strip()
df_collisions['charge_desc'] = df_collisions.charge_desc.str.strip()
df_collisions['hit_run_lvl'] = df_collisions.hit_run_lvl.str.strip()

## df_police calls

In [12]:
# Police Calls

# Change entries in col 'day_of_week' to represent the day of the week in words
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(1,'Monday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(2,'Tuesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(3,'Wednesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(4,'Thursday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(5,'Friday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(6,'Saturday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(7,'Sunday')

In [13]:
# Police Calls

# Date Time

# Creating seperate columns for Date and Time
date_time = df_police_calls['date_time'].str.split(' ', n=1, expand=True)
df_police_calls = df_police_calls.drop(['date_time'], axis=1)
df_police_calls['date'] = date_time[0]
df_police_calls['time'] = date_time[1]

# convert the 'Date' column to datetime format 
df_police_calls['date']= pd.to_datetime(df_police_calls['date']) 
df_police_calls['time']= pd.to_datetime(df_police_calls['time']) 

In [14]:
# Changing datatype of address_number_primary from int to str
df_police_calls['address_number_primary'] = df_police_calls.address_number_primary.astype(str)

In [15]:
# Police Calls

# dropping useless columns
drop_cols_calls = ['address_dir_primary', 'address_dir_intersecting','incident_num']
df_police_calls = df_police_calls.drop(drop_cols_calls,axis = 1)


In [16]:
# Police Calls

# String manipulations

# making everything lowercase
df_police_calls['address_road_primary']=df_police_calls['address_road_primary'].str.lower()
df_police_calls['address_road_intersecting']=df_police_calls['address_road_intersecting'].str.lower()
df_police_calls['address_sfx_primary'] = df_police_calls['address_sfx_primary'].str.lower()

# strip any extra spaces in string data
df_police_calls['address_road_primary']= df_police_calls.address_road_primary.str.strip()
df_police_calls['address_sfx_primary'] = df_police_calls.address_sfx_primary.str.strip()
df_police_calls['address_road_intersecting']= df_police_calls.address_road_intersecting.str.strip()
df_police_calls['call_type'] = df_police_calls.call_type.str.strip()
df_police_calls['disposition'] = df_police_calls.disposition.str.strip()

In [17]:
# Police calls

# Creating a single coulumn for primary address
df_police_calls['Primary address']= df_police_calls.address_number_primary + ' ' + df_police_calls.address_road_primary + ' '+ df_police_calls.address_sfx_primary

# Creating a single coulumn for intersecting address
df_police_calls['Intersecting address'] = df_police_calls.address_road_intersecting+ ' '+ df_police_calls.address_sfx_intersecting

# Dropping the newly useless columns
cols_drop = ['address_number_primary','address_road_primary','address_sfx_primary','address_sfx_intersecting','address_road_intersecting']
df_police_calls = df_police_calls.drop(cols_drop,axis = 1)

## df_police_stops

In [18]:
# Police Stops
drop_cols_stops = ['isstudent', 'gend_nc','agency','stop_id', 'gend']
df_police_stops = df_police_stops.drop(drop_cols_stops, axis = 1)

In [19]:
# Police Stops

# String manipulations

# making everything lowercase
df_police_stops['address_city'] = df_police_stops['address_city'].str.lower()


# Spliting the beat_name column into two columns such that beat name and beat number are seperate columns
name_num = df_police_stops['beat_name'].str.split(' ', n=1, expand=True)
df_police_stops = df_police_stops.drop(['beat_name'], axis=1)


print(df_police_stops)

# df_police_stops['beat_name'] = name_num[0] + name_num
# df_police_stops['beat_num'] = name_num[2]

              ori  exp_years   date_stop time_stop  stopduration  \
0       CA0371100         10  2018-07-01  00:01:37            30   
1       CA0371100         18  2018-07-01  00:03:34            10   
2       CA0371100          1  2018-07-01  00:05:43            15   
3       CA0371100          1  2018-07-01  00:05:43            15   
4       CA0371100          3  2018-07-01  00:19:06             5   
5       CA0371100          1  2018-07-01  00:03:00            15   
6       CA0371100         24  2018-07-01  00:24:02            20   
7       CA0371100          1  2018-07-01  00:31:19           120   
8       CA0371100          4  2018-07-01  00:33:19             3   
9       CA0371100          3  2018-07-01  00:11:00            20   
10      CA0371100          1  2018-07-01  00:34:49             5   
11      CA0371100          4  2018-07-01  00:38:46             6   
12      CA0371100          1  2018-07-01  00:37:08            10   
13      CA0371100          3  2018-07-01  00:42:

In [20]:
df_police_stops['perceived_limited_english'] = df_police_stops['perceived_limited_english'].replace(0,'no')
df_police_stops['perceived_limited_english'] = df_police_stops['perceived_limited_english'].replace(1,'yes')


df_police_stops['gender_nonconforming'] = df_police_stops['gender_nonconforming'].replace(0,'no')
df_police_stops['gender_nonconforming'] = df_police_stops['gender_nonconforming'].replace(1,'yes')

# Data Analysis & Results

In [21]:
df_police_stops.head()

,ori,exp_years,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,intersection,address_block,...,isschool,school_name,address_city,beat,pid,perceived_limited_english,perceived_age,perceived_gender,gender_nonconforming,perceived_lgbt
0,CA0371100,10,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",NaN,700.0,...,0,NaN,san diego,122,1,no,25,Male,no,No
1,CA0371100,18,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",I-5,NaN,...,0,NaN,san diego,121,1,no,25,Male,no,No
2,CA0371100,1,2018-07-01,00:05:43,15,1,10,Other,NaN,4400.0,...,0,NaN,san diego,822,1,no,30,Male,no,No
3,CA0371100,1,2018-07-01,00:05:43,15,1,10,Other,NaN,4400.0,...,0,NaN,san diego,822,2,no,30,Female,no,No
4,CA0371100,3,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",NaN,4800.0,...,0,NaN,san diego,614,1,no,23,Male,no,No


In [22]:
for col in df_police_stops.columns:
    
    print(col)

ori
exp_years
date_stop
time_stop
stopduration
stop_in_response_to_cfs
officer_assignment_key
assignment
intersection
address_block
land_mark
address_street
highway_exit
isschool
school_name
address_city
beat
pid
perceived_limited_english
perceived_age
perceived_gender
gender_nonconforming
perceived_lgbt


# Ethics & Privacy

# Conclusion & Discussion

## Geospatial tools
- Geojson: https://pypi.org/project/geojson/
    - https://www.datacamp.com/community/tutorials/geospatial-data-python
- GeoPandas: http://geopandas.org/

visualization tools: https://medium.com/@stallonejacob/d3-in-juypter-notebook-685d6dca75c8